# Final Team Project: Marketing Strategy Evaluation for New Menu Item
---

## Description:

This project aims to evaluate the effectiveness of three different marketing campaigns for a new menu item introduced by a fast-food chain. Using A/B testing, we will determine which promotion has the greatest impact on sales across various market locations. Our analysis will involve data cleaning, pre-processing, exploratory data analysis (EDA), and the application of data science methods using Python to derive actionable insights.

---

## Importing Required Libraries

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import matplotlib.pylab as plt